<a href="https://colab.research.google.com/github/nguyentuyetnhung/baitap25-9/blob/main/moneycnnipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

currency_info = {
    0: {
        "m·ªánh_gi√°": "100,000 VND",
        "m·∫∑t_tr∆∞·ªõc": "Ch·ªß t·ªãch H·ªì Ch√≠ Minh",
        "m·∫∑t_sau": "Nh√† s√†n B√°c H·ªì t·∫°i Ph·ªß Ch·ªß t·ªãch",
        "nƒÉm_ph√°t_h√†nh": "2004",
        "ch·∫•t_li·ªáu": "Polymer",
        "k√≠ch_th∆∞·ªõc": "144 x 65 mm",
        "m√†u_s·∫Øc_ch·ªß_ƒë·∫°o": "Xanh l√° c√¢y",
        "ƒë·∫∑c_ƒëi·ªÉm_n·ªïi_b·∫≠t": "H√¨nh b√≥ng ch√¨m, s·ªë ƒë·ªïi m√†u, y·∫øu t·ªë in n·ªïi",
        "an_ninh": "S·ªë 100000 ƒë·ªïi m√†u t·ª´ xanh sang v√†ng"
    },
    1: {
        "m·ªánh_gi√°": "200,000 VND",
        "m·∫∑t_tr∆∞·ªõc": "Ch·ªß t·ªãch H·ªì Ch√≠ Minh",
        "m·∫∑t_sau": "Qu√™ h∆∞∆°ng Kim Li√™n - Ngh·ªá An",
        "nƒÉm_ph√°t_h√†nh": "2006",
        "ch·∫•t_li·ªáu": "Polymer",
        "k√≠ch_th∆∞·ªõc": "148 x 65 mm",
        "m√†u_s·∫Øc_ch·ªß_ƒë·∫°o": "ƒê·ªè cam",
        "ƒë·∫∑c_ƒëi·ªÉm_n·ªïi_b·∫≠t": "H√¨nh ·∫©n, s·ªë ƒë·ªïi m√†u, y·∫øu t·ªë UV",
        "an_ninh": "S·ªë 200000 ƒë·ªïi m√†u t·ª´ ƒë·ªè sang xanh"
    },
    2: {
        "m·ªánh_gi√°": "500,000 VND",
        "m·∫∑t_tr∆∞·ªõc": "Ch·ªß t·ªãch H·ªì Ch√≠ Minh",
        "m·∫∑t_sau": "L√†ng Sen qu√™ B√°c",
        "nƒÉm_ph√°t_h√†nh": "2003",
        "ch·∫•t_li·ªáu": "Polymer",
        "k√≠ch_th∆∞·ªõc": "152 x 65 mm",
        "m√†u_s·∫Øc_ch·ªß_ƒë·∫°o": "T√≠m than",
        "ƒë·∫∑c_ƒëi·ªÉm_n·ªïi_b·∫≠t": "C·ª≠a s·ªï trong su·ªët, s·ªë ƒë·ªïi m√†u, h√¨nh ·∫©n",
        "an_ninh": "C·ª≠a s·ªï trong su·ªët h√¨nh b√¥ng sen"
    }
}

class_names = ['100000', '200000', '500000']

def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def load_data(data_dir):
    images = []
    labels = []

    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.exists(class_dir):
            print(f"Th∆∞ m·ª•c {class_dir} kh√¥ng t·ªìn t·∫°i")
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            try:

                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224, 224))
                image = image.astype('float32') / 255.0

                images.append(image)
                labels.append(class_names.index(class_name))
            except Exception as e:
                print(f"L·ªói khi ƒë·ªçc ·∫£nh {img_path}: {e}")

    return np.array(images), np.array(labels)


def train_model():

    if os.path.exists('/content/money_cnn_model.h5'):
        print("Model ƒë√£ t·ªìn t·∫°i, b·ªè qua training...")
        return tf.keras.models.load_model('/content/money_cnn_model.h5')


    possible_paths = [
         "/content/drive/MyDrive/tiencnn"
    ]

    data_dir = None
    for path in possible_paths:
        if os.path.exists(path):
            data_dir = path
            break

    if data_dir is None:
        print("Kh√¥ng t√¨m th·∫•y d·ªØ li·ªáu training. T·∫°o model m·ªõi...")
        return create_model()

    print(f"ƒêang load d·ªØ li·ªáu t·ª´: {data_dir}")

    # Load d·ªØ li·ªáu
    X, y = load_data(data_dir)

    if len(X) == 0:
        print("Kh√¥ng c√≥ d·ªØ li·ªáu ƒë·ªÉ training. T·∫°o model m·ªõi...")
        return create_model()

    y = tf.keras.utils.to_categorical(y, 3)


    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2
    )

    # T·∫°o model
    model = create_model()


    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        '/content/money_cnn_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )

    print("B·∫Øt ƒë·∫ßu training model...")
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=32),
        epochs=50,
        validation_data=(X_val, y_val),
        callbacks=[checkpoint],
        verbose=1
    )

    # V·∫Ω bi·ªÉu ƒë·ªì training
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

    print("Training ho√†n th√†nh! Model ƒë√£ ƒë∆∞·ª£c l∆∞u v√†o money_cnn_model.h5")
    try:
        os.system('cp /content/money_cnn_model.h5 /content/drive/MyDrive/money_cnn_model.h5')
        print("ƒê√£ l∆∞u backup model l√™n Google Drive")
    except:
        print("Kh√¥ng th·ªÉ l∆∞u backup l√™n Google Drive")

    return model

# Load ho·∫∑c train model
print("ƒêang kh·ªüi t·∫°o model...")
model = train_model()

def preprocess_image(image):
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (224, 224))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# H√†m d·ª± ƒëo√°n
def predict_currency(image):
    processed_image = preprocess_image(image)

    prediction = model.predict(processed_image, verbose=0)
    class_idx = np.argmax(prediction[0])
    confidence = prediction[0][class_idx] * 100

    currency_data = currency_info[class_idx]


    result_html = f"""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 15px; color: white; margin-bottom: 20px;">
        <h2 style="text-align: center; margin-bottom: 20px;">üéâ K·∫æT QU·∫¢ NH·∫¨N DI·ªÜN üéâ</h2>
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
            <h3 style="color: #ffd700; margin-bottom: 10px;">M·ªánh gi√°: {currency_data['m·ªánh_gi√°']}</h3>
            <p style="margin: 5px 0;"><strong>ƒê·ªô tin c·∫≠y:</strong> {confidence:.2f}%</p>
        </div>
    </div>
    """

    # T·∫°o th√¥ng tin chi ti·∫øt
    details_html = f"""
    <div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 20px; border-radius: 15px; color: white;">
        <h3 style="text-align: center; margin-bottom: 15px;">üìã TH√îNG TIN CHI TI·∫æT</h3>
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px;">
            <div><strong>M·∫∑t tr∆∞·ªõc:</strong> {currency_data['m·∫∑t_tr∆∞·ªõc']}</div>
            <div><strong>M·∫∑t sau:</strong> {currency_data['m·∫∑t_sau']}</div>
            <div><strong>NƒÉm ph√°t h√†nh:</strong> {currency_data['nƒÉm_ph√°t_h√†nh']}</div>
            <div><strong>Ch·∫•t li·ªáu:</strong> {currency_data['ch·∫•t_li·ªáu']}</div>
            <div><strong>K√≠ch th∆∞·ªõc:</strong> {currency_data['k√≠ch_th∆∞·ªõc']}</div>
            <div><strong>M√†u s·∫Øc:</strong> {currency_data['m√†u_s·∫Øc_ch·ªß_ƒë·∫°o']}</div>
        </div>
        <div style="margin-top: 15px;">
            <strong>ƒê·∫∑c ƒëi·ªÉm n·ªïi b·∫≠t:</strong> {currency_data['ƒë·∫∑c_ƒëi·ªÉm_n·ªïi_b·∫≠t']}
        </div>
        <div style="margin-top: 10px;">
            <strong>Y·∫øu t·ªë an ninh:</strong> {currency_data['an_ninh']}
        </div>
    </div>
    """

    fig, ax = plt.subplots(figsize=(8, 4))
    bars = ax.bar(class_names, prediction[0] * 100, color=['#ff6b6b', '#4ecdc4', '#45b7d1'])
    ax.set_ylabel('Confidence (%)')
    ax.set_title('ƒê·ªò TIN C·∫¨Y D·ª∞ ƒêO√ÅN')
    ax.set_ylim(0, 100)

    for bar, value in zip(bars, prediction[0] * 100):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()

    return result_html, details_html, fig
with gr.Blocks(theme=gr.themes.Soft(), title="Nh·∫≠n Di·ªán Ti·ªÅn Vi·ªát Nam") as demo:
    gr.Markdown(
        """
        # üè¶ NH·∫¨N DI·ªÜN TI·ªÄN VI·ªÜT NAM
        **·ª®ng d·ª•ng s·ª≠ d·ª•ng CNN ƒë·ªÉ nh·∫≠n di·ªán 3 m·ªánh gi√°: 100,000VND - 200,000VND - 500,000VND**
        """
    )

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(
                label="üì∑ T·∫¢I L√äN H√åNH ·∫¢NH T·ªú TI·ªÄN",
                type="pil",
                height=300
            )
            predict_btn = gr.Button("üéØ NH·∫¨N DI·ªÜN", variant="primary", size="lg")

            gr.Markdown("### üìù H∆∞·ªõng d·∫´n s·ª≠ d·ª•ng:")
            gr.Markdown("""
            - Ch·ª•p ·∫£nh r√µ n√©t t·ªù ti·ªÅn c·∫ßn nh·∫≠n di·ªán
            - ·∫¢nh n√™n c√≥ ƒë·ªß √°nh s√°ng v√† t·∫≠p trung v√†o t·ªù ti·ªÅn
            - C√°c m·ªánh gi√° h·ªó tr·ª£: 100,000VND, 200,000VND, 500,000VND
            - Model ƒë√£ ƒë∆∞·ª£c training v√† l∆∞u v√†o **money_cnn_model.h5**
            """)

        with gr.Column():
            result_output = gr.HTML(label="üéâ K·∫æT QU·∫¢")
            details_output = gr.HTML(label="üìã TH√îNG TIN CHI TI·∫æT")
            plot_output = gr.Plot(label="üìä BI·ªÇU ƒê·ªí ƒê·ªò TIN C·∫¨Y")

    predict_btn.click(
        fn=predict_currency,
        inputs=image_input,
        outputs=[result_output, details_output, plot_output]
    )


    with gr.Accordion("‚ÑπÔ∏è TH√îNG TIN MODEL", open=False):
        gr.Markdown(f"""
        **Model CNN cho nh·∫≠n di·ªán ti·ªÅn Vi·ªát Nam:**
        - **Architecture:** CNN v·ªõi 4 l·ªõp convolutional
        - **Input size:** 224x224x3
        - **Output:** 3 classes (100K, 200K, 500K)
        - **Model file:** money_cnn_model.h5
        - **Status:** {'ƒê√£ training' if os.path.exists('/content/money_cnn_model.h5') else 'Ch∆∞a training'}
        """)

if __name__ == "__main__":
    demo.launch(
        share=True,
        debug=True,
        show_error=True
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ƒêang kh·ªüi t·∫°o model...
ƒêang load d·ªØ li·ªáu t·ª´: /content/drive/MyDrive/tiencnn
Th∆∞ m·ª•c /content/drive/MyDrive/tiencnn/100000 kh√¥ng t·ªìn t·∫°i


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


B·∫Øt ƒë·∫ßu training model...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 3s/step - accuracy: 0.4071 - loss: 1.4285
Epoch 1: val_accuracy improved from -inf to 0.52174, saving model to /content/money_cnn_model.h5


3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 16s 4s/step - accuracy: 0.4192 - loss: 1.4797 - val_accuracy: 0.5217 - val_loss: 0.8224
Epoch 2/50
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 3s/step - accuracy: 0.5439 - loss: 0.8006
Epoch 2: val_accuracy did not improve from 0.52174
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 13s 4s/step - accuracy: 0.5385 - loss: 0.7970 - val_accuracy: 0.4783 - val_loss: 0.7282
Epoch 3/50
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 5s/step - accuracy: 0.5527 - loss: 0.7524
Epoch 3: val_accuracy improved from 0.52174 to 0.95652, saving model to /content/money_cnn_model.h5


3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 17s 6s/step - accuracy: 0.5478 - loss: 0.7563 - val_accuracy: 0.9565 - val_loss: 0.6557
Epoch 4/50
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 4s/step - accuracy: 0.5282 - loss: 0.8217
Epoch 4: val_accuracy did not improve from 0.95652
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 18s 4s/step - accuracy: 0.5267 - loss: 0.8262 - val_accuracy: 0.5217 - val_loss: 0.6985
Epoch 5/50
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 4s/step - accuracy: 0.7410 - loss: 0.6710
Epoch 5: val_accuracy did not improve from 0.95652
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 12s 4s/step - accuracy: 0.7252 - loss: 0.6716 - val_accuracy: 0.9565 - val_loss: 0.5825
Epoch 6/50
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 3s/step - accuracy: 0.5772 - loss: 0.6210
Epoch 6: val_accuracy improved from 0.95652 to 1.00000, saving model to /con

3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 13s 4s/step - accuracy: 0.5885 - loss: 0.6145 - val_accuracy: 1.0000 - val_loss: 0.4396
Epoch 7/50
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 3s/step - accuracy: 0.9114 - loss: 0.3857
Epoch 7: val_accuracy did not improve from 1.00000
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 13s 4s/step - accuracy: 0.9085 - loss: 0.3838 - val_accuracy: 0.9565 - val_loss: 0.1538
Epoch 8/50
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 3s/step - accuracy: 0.9208 - loss: 0.2179
Epoch 8: val_accuracy did not improve from 1.00000
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 13s 4s/step - accuracy: 0.9184 - loss: 0.2188 - val_accuracy: 0.9565 - val_loss: 0.1787
Epoch 9/50
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 3s/step - accuracy: 0.9700 - loss: 0.1481
Epoch 9: val_accuracy did not improve from 1.00000
3/3 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ

Training ho√†n th√†nh! Model ƒë√£ ƒë∆∞·ª£c l∆∞u v√†o money_cnn_model.h5
ƒê√£ l∆∞u backup model l√™n Google Drive
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1533046a385e37d170.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
